# Monitoring Regression Model Performance Metrics

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/whylabs/whylogs/blob/mainline/python/examples/integrations/writers/Writing_Regression_Performance_Metrics_to_WhyLabs.ipynb)

In this tutorial, we'll show how you can log performance metrics of your ML Model with whylogs, and how to send it to your dashboard at Whylabs Platform. We'll follow a regression use case, where we're predicting the temperature of a given location based on metereological features.

We will:

- Download Weather Data for 7 days
- Log daily input features with whylogs
- Log daily regression performance metrics with whylogs
- Write logged profiles to WhyLabs' dashboard
- Show performance summary at WhyLabs

## Installing whylogs

First, let's install __whylogs__. Since we want to write to WhyLabs, we'll install the __whylabs__ extra. Additionally, we'll use the __datasets__ module, so let's install it as well:

In [1]:
# %pip install whylogs[whylabs,datasets]

## 🌤️ The Data - Weather Forecast Dataset

In this example, we will use the Weather Forecast Dataset, using whylogs' __Datasets__ module.

This dataset contains several meteorological features at a particular place (defined by latitude and longitude features) and time. The task is to predict the temperature based on the input features.

The original data is described in [Shifts: A Dataset of Real Distributional Shift Across Multiple Large-Scale Tasks](https://arxiv.org/pdf/2107.07455.pdf), by **Malinin, Andrey, et al.**, and was further transformed to compose the current dataset. You can have more information about the resulting dataset and how to use it at https://whylogs.readthedocs.io/en/latest/datasets/weather.html.

### Downloading the data into daily batches

Let's download 7 batches with 7 days worth of data, corresponding to the last 7 days. We can use directly the __datasets__ module for that.

In [1]:
from whylogs.datasets import Weather
from datetime import datetime, timezone, timedelta
dataset = Weather()

start_timestamp = datetime.now(timezone.utc) - timedelta(days=7)
dataset.set_parameters(inference_start_timestamp=start_timestamp)

daily_batches = dataset.get_inference_data(number_batches=7)

#batches is an iterator, so let's get the list for this
daily_batches = list(daily_batches)

Since in this example we're mainly concerned with regression metrics, let's select a subset of the available features, for simplicity.

__meta_climate__, __meta_latitude__, __meta_longitude__ will be our input features, while __prediction_temperature__ is the predicted feature given by a trained ML model and the __temperature__ feature is our target.

Let's take a look at the data for the first day:

In [2]:
columns = ["meta_climate","meta_latitude","meta_longitude","prediction_temperature","temperature"]
df = daily_batches[0].data[columns]

df.head()

,meta_climate,meta_latitude,meta_longitude,prediction_temperature,temperature
date,,,,,
2022-09-07 00:00:00+00:00,mild temperate,38.891300,-6.821330,9.163181,10.0
2022-09-07 00:00:00+00:00,tropical,12.216667,109.216667,26.220221,27.0
2022-09-07 00:00:00+00:00,dry,37.991699,-101.746002,13.178478,15.0
2022-09-07 00:00:00+00:00,mild temperate,-23.333599,-51.130100,23.255124,25.0
2022-09-07 00:00:00+00:00,mild temperate,-23.479445,-52.012222,27.851674,32.0


## ✔️ Setting the Environment Variables

In order to send our profile to WhyLabs, let's first set up an account. You can skip this if you already have an account and a model set up.

We will need three pieces of information:

- API token
- Organization ID
- Dataset ID (or model-id)

Go to https://whylabs.ai/free and grab a free account. You can follow along with the examples if you wish, but if you’re interested in only following this demonstration, you can go ahead and skip the quick start instructions.

After that, you’ll be prompted to create an API token. Once you create it, copy and store it locally. The second important information here is your org ID. Take note of it as well. After you get your API Token and Org ID, you can go to https://hub.whylabsapp.com/models to see your projects dashboard. You can create a new project and take note of it's ID (if it's a model project it will look like `model-xxxx`).

In [3]:
import getpass
import os

# set your org-id here - should be something like "org-xxxx"
print("Enter your WhyLabs Org ID") 
os.environ["WHYLABS_DEFAULT_ORG_ID"] = input()

# set your datased_id (or model_id) here - should be something like "model-xxxx"
print("Enter your WhyLabs Dataset ID")
os.environ["WHYLABS_DEFAULT_DATASET_ID"] = input()


# set your API key here
print("Enter your WhyLabs API key")
os.environ["WHYLABS_API_KEY"] = getpass.getpass()
print("Using API Key ID: ", os.environ["WHYLABS_API_KEY"][0:10])

Enter your WhyLabs Org ID
Enter your WhyLabs Dataset ID
Enter your WhyLabs API key
Using API Key ID:  z8fYdnQwHr


## 📊 Profiling the Data + Sending to WhyLabs

Traditionally, data is logged by calling `why.log()`. In this case, we'll use `why.log_regression_metrics()`, which will log data all the same, but additionally it will compute regression metrics and add them to your results.

`log_regression_metrics` takes the complete dataframe as input (with input/output features, as well as your prediction column and target column). We also have to define which column is our target (in this case, `temperature`) and which is our prediction column (`prediction_temperature` in this case).

Once the profile is logged, we can set it's timestamp for the proper day as given by our batch's timestamp.

Now that we have properly timestamped profiles with regression metrics, we can use the `writer` method to send it to WhyLabs:

In [4]:
from whylogs.api.writer.whylabs import WhyLabsWriter
import whylogs as why

columns = ["meta_climate","meta_latitude","meta_longitude","prediction_temperature","temperature"]
for batch in daily_batches:
    dataset_timestamp = batch.timestamp

    df = batch.data[columns]
    print("logging data for date {}".format(dataset_timestamp))
    results = why.log_regression_metrics(df, target_column = "temperature", prediction_column = "prediction_temperature")

    profile = results.profile()
    profile.set_dataset_timestamp(dataset_timestamp)
    print("writing profiles to whylabs...")
    results.writer("whylabs").write()

logging data for date 2022-09-07 00:00:00+00:00
writing profiles to whylabs...
logging data for date 2022-09-08 00:00:00+00:00
writing profiles to whylabs...
logging data for date 2022-09-09 00:00:00+00:00
writing profiles to whylabs...
logging data for date 2022-09-10 00:00:00+00:00
writing profiles to whylabs...
logging data for date 2022-09-11 00:00:00+00:00
writing profiles to whylabs...
logging data for date 2022-09-12 00:00:00+00:00
writing profiles to whylabs...
logging data for date 2022-09-13 00:00:00+00:00
writing profiles to whylabs...


And that's it! You just sent your profiles to WhyLabs.

At your model's dashboard, you should see the model metrics for the last seven days. For regression, the displayed metrics are:

- Total output and input count
- Mean Squared Error
- Mean Absolute Error
- Root Mean Squared Error

![alt text](images/regression_metrics.png)